In [1]:
import numpy as np, pandas as pd, xarray as xr
from pathlib import Path
import datetime, networkx as nx, yaml, warnings, sys, logging, shutil, papermill, re
from disjoint_set import DisjointSet
from helper import singleglob, nxrender, Step, dict_merge, PdfWriter, DictWriter, TableWriter, TextWriter
import matplotlib.pyplot as plt
import matplotlib as mpl, seaborn as sns
from tqdm.notebook import tqdm
from typing import Literal


In [2]:
folder = Path("/home/julienb/Documents/Data/Luisa/")
scripts =  ["task_graph.ipynb", "events.ipynb"] #["task_graph.ipynb", "events.ipynb", "plot_rt_mt.ipynb"]
selection: Literal["base",  "subject", "session", "existing"] = "session"
copy: Literal["force", "notexisting"]= "notexisting"
exec: Literal["force", "notexisting", "notsucess", "erase"]= "notexisting"

In [3]:
dfs = []
for i, script in enumerate(scripts):
    df = pd.DataFrame()
    if selection=="base":
        df["base_path"] = list(folder.glob("analysis"))
    elif selection=="subject":
        df["base_path"] = list(folder.glob("*/analysis"))
    elif selection=="session":
        df["base_path"] = list(folder.glob("**/Session--*/analysis"))
    elif selection=="existing":
        df["base_path"] = [f.parent for f in folder.glob(f"**/{script}")]
    df.insert(0, "quick_path", df["base_path"].apply(lambda p: str(Path(*p.parts[-3:-1]))))
    dfs.append(df.assign(script=Path(script), script_index=i, script_path=df.apply(lambda row: (row["base_path"]/script), axis=1)))
tasks = pd.concat(dfs).sort_values(["base_path", "script_index"])
tasks

,quick_path,base_path,script,script_index,script_path
119,opto_ALLTRIALS/Session--Rat101_20240911,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
119,opto_ALLTRIALS/Session--Rat101_20240911,/home/julienb/Documents/Data/Luisa/Rat101/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat101/opto...
118,opto_ALLTRIALS/Session--Rat101_20240912,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
118,opto_ALLTRIALS/Session--Rat101_20240912,/home/julienb/Documents/Data/Luisa/Rat101/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat101/opto...
112,opto_CueRT/Session--Rat101_20240828,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...
...,...,...,...,...,...
20,opto_MT/Session--Rat246_20240805,/home/julienb/Documents/Data/Luisa/Rat246/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat246/opto...
21,opto_MT/Session--Rat246_20240806,/home/julienb/Documents/Data/Luisa/Rat246/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...
21,opto_MT/Session--Rat246_20240806,/home/julienb/Documents/Data/Luisa/Rat246/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat246/opto...
19,opto_MT/Session--Rat246_20240807,/home/julienb/Documents/Data/Luisa/Rat246/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...


In [4]:

if copy=="force": tasks["copy"] = True 
elif copy=="notexisting": tasks["copy"] = tasks["script_path"].apply(lambda p: not p.exists())
else: raise Exception(f"Unknown copy param {copy}")

if exec=="force": 
    tasks["exec"] = True
    tasks["erase"] = True
elif exec=="notexisting": 
    tasks["exec"] = tasks["script_path"].apply(lambda p: not p.with_suffix("").exists()) 
    tasks["erase"] = False
elif exec=="notsucess": 
    tasks["exec"] = tasks["script_path"].apply(lambda p: not (p.with_suffix("")/"notebook.html").exists())
    tasks["erase"] = False
elif exec=="erase":
    tasks["exec"] = False
    tasks["erase"] = True
else: raise Exception(f"Unknown exec param {exec}")

display(tasks.groupby(["copy", "exec", "script"])["base_path"].count().rename("count").to_frame().reset_index())
tasks


,copy,exec,script,count
0,False,False,events.ipynb,142
1,False,False,task_graph.ipynb,142
2,False,True,events.ipynb,41
3,False,True,task_graph.ipynb,41


,quick_path,base_path,script,script_index,script_path,copy,exec,erase
119,opto_ALLTRIALS/Session--Rat101_20240911,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False,False
119,opto_ALLTRIALS/Session--Rat101_20240911,/home/julienb/Documents/Data/Luisa/Rat101/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False,False
118,opto_ALLTRIALS/Session--Rat101_20240912,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False,False
118,opto_ALLTRIALS/Session--Rat101_20240912,/home/julienb/Documents/Data/Luisa/Rat101/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False,False
112,opto_CueRT/Session--Rat101_20240828,/home/julienb/Documents/Data/Luisa/Rat101/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat101/opto...,False,False,False
...,...,...,...,...,...,...,...,...
20,opto_MT/Session--Rat246_20240805,/home/julienb/Documents/Data/Luisa/Rat246/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat246/opto...,False,False,False
21,opto_MT/Session--Rat246_20240806,/home/julienb/Documents/Data/Luisa/Rat246/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...,False,False,False
21,opto_MT/Session--Rat246_20240806,/home/julienb/Documents/Data/Luisa/Rat246/opto...,events.ipynb,1,/home/julienb/Documents/Data/Luisa/Rat246/opto...,False,False,False
19,opto_MT/Session--Rat246_20240807,/home/julienb/Documents/Data/Luisa/Rat246/opto...,task_graph.ipynb,0,/home/julienb/Documents/Data/Luisa/Rat246/opto...,False,False,False


In [5]:
copy_df = tasks.loc[tasks["copy"]]
progress = tqdm(copy_df[["base_path", "script"]].iterrows(), total=len(copy_df.index))
for _, (p,s) in progress:
    shutil.copy(s, p/s)

0it [00:00, ?it/s]

In [6]:
erase_df = tasks.loc[tasks["erase"]]
progress = tqdm(erase_df[["base_path", "script"]].iterrows(), total=len(erase_df.index))
for _, (p,s) in progress:
    if (p/s.stem).exists():
        shutil.rmtree(p/s.stem)

0it [00:00, ?it/s]

In [7]:
%env PYDEVD_DISABLE_FILE_VALIDATION=1
exec_df = tasks.loc[tasks["exec"]]
base_path_grp = exec_df.groupby("base_path")
progress = tqdm(base_path_grp, total=len(base_path_grp))
errors=[]

for p, grp in progress:
    quick_path = grp["quick_path"].iat[0]
    progress.set_description_str(quick_path)
    grp = grp.sort_values("script_index")
    skip=False
    for script in grp["script"]:
        if not skip:
            progress.set_postfix_str(f"n_prev_err={len(errors)}, Executing {script}")
            (p/script.stem).mkdir(exist_ok=True, parents=True)
            try:
                papermill.execute_notebook(p/script, p/script.stem/"notebook.ipynb", cwd=p, progress_bar=False,  log_ouput=True)
            except Exception as e:
                errors.append((quick_path, p, script, e))
                e.add_note(f"During execution of {script} in {p}")
                # print(f'Error in {p}')
                display(print(e))
                skip=True
                # raise
                
if len(errors) > 0:
    errors_df = pd.DataFrame(errors, columns=["quick_path", "base_path", "script", "exception"])
    errors_df["err_message"] = errors_df["exception"].apply(lambda e: re.findall(r'Exception:.*$', str(e)))
    display(errors_df[["quick_path", "script", "err_message"]])
    raise ExceptionGroup("Errors during execution", [e[-1] for e in errors])

env: PYDEVD_DISABLE_FILE_VALIDATION=1


  0%|          | 0/41 [00:00<?, ?it/s]

Error in /home/julienb/Documents/Data/Luisa/Rat230/opto_Fastest/Session--Rat230_20240911/analysis

---------------------------------------------------------------------------
Exception encountered at "In [16]":
---------------------------------------------------------------------------
Exception                                 Traceback (most recent call last)
Cell In[16], line 5
      2 shift_using = "cue"
      4 if not (merge_data["num_events"].sel(event=shift_using) == merge_data["num_events"].sel(source=base_time_source, event=shift_using)).all():
----> 5     raise Exception(f"Problem not same number of {shift_using} event")
      7 def get_shift(df, target_df):
      8     df = df["df"]

Exception: Problem not same number of cue event

Error in /home/julienb/Documents/Data/Luisa/Rat230/opto_Fastest/Session--Rat230_20240912/analysis

---------------------------------------------------------------------------
Exception encountered at "In [16]":
-------------------------------------